# Supply Chain Forecasting: Advanced Gradient Boosting Approach

**Objective:** Predict cumulative raw material deliveries for January-May 2025

**Strategy:**
- Window-based feature engineering (not sequence-based)
- Rich historical aggregates from multiple data sources
- LightGBM with quantile regression (α=0.2)
- Time-aware validation and monotonicity enforcement

## 1. Setup and Configuration

In [19]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import pandas as pd
import numpy as np
import warnings
from datetime import datetime, timedelta
from typing import Dict, Tuple
import gc

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("✅ Environment configured")

✅ Environment configured


## 2. Data Loading and Initial Preprocessing

In [20]:
print("📂 Loading datasets...")

# Load core datasets
receivals = pd.read_csv('data/kernel/receivals.csv', parse_dates=['date_arrival'])
purchase_orders = pd.read_csv('data/kernel/purchase_orders.csv', parse_dates=['delivery_date'])
materials = pd.read_csv('data/extended/materials.csv')
prediction_mapping = pd.read_csv('data/prediction_mapping.csv')

# Parse dates in mapping
prediction_mapping['forecast_start_date'] = pd.to_datetime(prediction_mapping['forecast_start_date'], utc=True)
prediction_mapping['forecast_end_date'] = pd.to_datetime(prediction_mapping['forecast_end_date'], utc=True)

print(f"Receivals: {receivals.shape}")
print(f"Purchase Orders: {purchase_orders.shape}")
print(f"Materials: {materials.shape}")
print(f"Prediction Mapping: {prediction_mapping.shape}")

# Data quality: remove invalid entries
receivals = receivals[receivals['net_weight'] > 0].copy()
purchase_orders = purchase_orders[purchase_orders['quantity'] > 0].copy()

# Convert pounds to kg (unit_id 43 = pounds)
LBS_TO_KG = 0.453592
mask_lbs = purchase_orders['unit_id'] == 43
purchase_orders.loc[mask_lbs, 'quantity'] *= LBS_TO_KG
purchase_orders.loc[mask_lbs, 'unit_id'] = 40  # kg unit_id

print("✅ Data loaded and cleaned")

📂 Loading datasets...
Receivals: (122590, 10)
Purchase Orders: (33171, 12)
Materials: (1218, 6)
Prediction Mapping: (30450, 4)
✅ Data loaded and cleaned


## 3. Create Product-Material Mapping

In [21]:
# Build rm_id to product_id mapping from materials
product_to_rm = materials[['product_id', 'rm_id']].drop_duplicates()
product_to_rm = product_to_rm.dropna(subset=['product_id', 'rm_id'])

# Link purchase orders to raw materials
po_with_rm = purchase_orders.merge(
    product_to_rm,
    on='product_id',
    how='left'
)
po_with_rm = po_with_rm.dropna(subset=['rm_id'])

print(f"✅ Linked {len(po_with_rm)} purchase orders to raw materials")

✅ Linked 340195 purchase orders to raw materials


## 4. Daily Aggregations

In [22]:
# Aggregate receivals and POs to daily level per material

print("🔄 Creating daily aggregations...")

# Ensure datetime (without timezone issues)
receivals['date_arrival'] = pd.to_datetime(receivals['date_arrival'], utc=True).dt.tz_localize(None)
po_with_rm['delivery_date'] = pd.to_datetime(po_with_rm['delivery_date'], utc=True).dt.tz_localize(None)

# Daily receivals per material (use normalize() instead of dt.date)
daily_receivals = receivals.copy()
daily_receivals['date'] = daily_receivals['date_arrival'].dt.normalize()

daily_receivals = daily_receivals.groupby(['date', 'rm_id']).agg({
    'net_weight': ['sum', 'count', 'mean']
}).reset_index()

daily_receivals.columns = ['date', 'rm_id', 'weight_sum', 'delivery_count', 'weight_mean']

# Daily purchase orders per material
daily_pos = po_with_rm.copy()
daily_pos['date'] = daily_pos['delivery_date'].dt.normalize()

daily_pos = daily_pos.groupby(['date', 'rm_id']).agg({
    'quantity': ['sum', 'count']
}).reset_index()

daily_pos.columns = ['date', 'rm_id', 'po_quantity', 'po_count']

print(f"✅ Daily receivals: {daily_receivals.shape}")
print(f"   Date range: {daily_receivals['date'].min()} to {daily_receivals['date'].max()}")
print(f"✅ Daily POs: {daily_pos.shape}")
print(f"   Date range: {daily_pos['date'].min()} to {daily_pos['date'].max()}")

🔄 Creating daily aggregations...
✅ Daily receivals: (41906, 5)
   Date range: 2004-06-15 00:00:00 to 2024-12-19 00:00:00
✅ Daily POs: (43996, 4)
   Date range: 2002-04-29 00:00:00 to 2025-06-29 00:00:00


## 5. Advanced Feature Engineering Functions

In [23]:
def calculate_window_receivals(df: pd.DataFrame, start_date: datetime, 
                                end_date: datetime, rm_id: int) -> float:
    """Calculate total receivals in a window using vectorized operations"""
    mask = (
        (df['rm_id'] == rm_id) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    )
    return df.loc[mask, 'weight_sum'].sum()

def calculate_historical_stats(daily_df: pd.DataFrame, ref_date: datetime, 
                                rm_id: int, lookback_days: int) -> Dict:
    """Calculate historical statistics for a material"""
    start = ref_date - timedelta(days=lookback_days)
    mask = (
        (daily_df['rm_id'] == rm_id) &
        (daily_df['date'] >= start) &
        (daily_df['date'] < ref_date)
    )
    
    subset = daily_df.loc[mask]
    
    if len(subset) == 0:
        return {
            'sum': 0.0,
            'mean': 0.0,
            'std': 0.0,
            'count': 0
        }
    
    return {
        'sum': subset['weight_sum'].sum(),
        'mean': subset['weight_sum'].mean(),
        'std': subset['weight_sum'].std() if len(subset) > 1 else 0.0,
        'count': len(subset)
    }

print("✅ Feature engineering functions defined")

✅ Feature engineering functions defined


## 5.5 Debug: Check Data Availability

In [24]:
print("🔍 Debugging data availability...")
print(f"\nDaily receivals shape: {daily_receivals.shape}")
print(f"Daily receivals date range: {daily_receivals['date'].min()} to {daily_receivals['date'].max()}")
print(f"Unique materials in receivals: {daily_receivals['rm_id'].nunique()}")
print(f"\nDaily POs shape: {daily_pos.shape}")
print(f"Daily POs date range: {daily_pos['date'].min()} to {daily_pos['date'].max()}")
print(f"Unique materials in POs: {daily_pos['rm_id'].nunique()}")

# Check if dates are before 2025
receivals_before_2025 = daily_receivals[daily_receivals['date'] < pd.to_datetime('2025-01-01')]
print(f"\nReceival rows before 2025: {len(receivals_before_2025)}")

# Top 10 materials
top_10_materials = daily_receivals.groupby('rm_id').size().sort_values(ascending=False).head(10)
print(f"\nTop 10 materials by receival frequency:")
print(top_10_materials)

# Sample a specific material to test
test_rm = top_10_materials.index[0]
test_data = daily_receivals[daily_receivals['rm_id'] == test_rm]
print(f"\nTest material {test_rm}:")
print(f"  Receivals: {len(test_data)}")
print(f"  Date range: {test_data['date'].min()} to {test_data['date'].max()}")
print(f"  Total weight: {test_data['weight_sum'].sum()}")

# Test calculate_window_receivals function
test_start = datetime(2020, 1, 1)
test_end = datetime(2020, 1, 30)
test_target = calculate_window_receivals(daily_receivals, test_start, test_end, test_rm)
print(f"\nTest window calculation for {test_rm} (2020-01-01 to 2020-01-30): {test_target}")


🔍 Debugging data availability...

Daily receivals shape: (41906, 5)
Daily receivals date range: 2004-06-15 00:00:00 to 2024-12-19 00:00:00
Unique materials in receivals: 203

Daily POs shape: (43996, 4)
Daily POs date range: 2002-04-29 00:00:00 to 2025-06-29 00:00:00
Unique materials in POs: 202

Receival rows before 2025: 41906

Top 10 materials by receival frequency:
rm_id
2130.0    2940
2160.0    2450
2142.0    2312
2140.0    2172
2134.0    1787
2132.0    1735
2135.0    1562
2131.0    1518
2144.0    1515
1903.0    1512
dtype: int64

Test material 2130.0:
  Receivals: 2940
  Date range: 2011-11-15 00:00:00 to 2024-12-19 00:00:00
  Total weight: 351244347.0

Test window calculation for 2130.0 (2020-01-01 to 2020-01-30): 2766190.0


## 6-8. Optimized Training Data Generation with Features and Targets

In [25]:
# Give MORE weight to recent data, LESS weight to old data

print("🔄 Generating training dataset with temporal weighting...")

# Use full history (2016-2024) ma con pesi decrescenti verso il passato
TRAINING_YEARS = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
WINDOW_STEPS = [1, 3, 5, 7, 10, 14, 21, 30, 45, 60, 75, 90, 105, 120, 151]

# Define temporal weights (exponential increase towards 2024)
year_weights = {
    2016: 0.3,
    2017: 0.4,
    2018: 0.6,
    2019: 0.8,
    2020: 1.0,
    2021: 1.3,
    2022: 1.6,
    2023: 1.9,
    2024: 2.5   # 2024 è 8x più importante di 2016
}

print("Temporal weights:")
for year in TRAINING_YEARS:
    print(f"  {year}: {year_weights[year]:.1f}x")

# Get top materials
top_materials = (
    daily_receivals.groupby('rm_id')
    .size()
    .sort_values(ascending=False)
    .head(120)
    .index.tolist()
)

print(f"\nUsing {len(top_materials)} materials from years {TRAINING_YEARS}")

training_data_list = []
sample_weights_list = []
year_totals = {}

for year in TRAINING_YEARS:
    print(f"  Processing year {year} (weight={year_weights[year]:.1f}x)...")
    year_start = datetime(year, 1, 1)
    year_total = 0
    year_weight = year_weights[year]
    
    for rm_id in top_materials:
        for window_days in WINDOW_STEPS:
            start_date = year_start
            end_date = year_start + timedelta(days=window_days - 1)
            
            if end_date >= datetime(2025, 1, 1):
                continue
            
            target = calculate_window_receivals(
                daily_receivals, start_date, end_date, rm_id
            )
            
            if target <= 0:
                continue
            
            row = {
                'rm_id': rm_id,
                'start_date': start_date,
                'end_date': end_date,
                'window_days': window_days,
                'end_month': end_date.month,
                'end_quarter': (end_date.month - 1) // 3 + 1,
                'end_dayofyear': end_date.timetuple().tm_yday,
                'is_q1': 1 if end_date.month <= 3 else 0,
                'target': target
            }
            
            # Historical features
            ref_date = start_date
            rec_stats = calculate_historical_stats(daily_receivals, ref_date, rm_id, 30)
            row['rec_hist_30d_sum'] = rec_stats['sum']
            row['rec_hist_30d_mean'] = rec_stats['mean']
            
            po_hist_mask = (
                (daily_pos['rm_id'] == rm_id) &
                (daily_pos['date'] >= ref_date - timedelta(days=30)) &
                (daily_pos['date'] < ref_date)
            )
            row['po_hist_30d_sum'] = daily_pos.loc[po_hist_mask, 'po_quantity'].sum()
            
            po_window_mask = (
                (daily_pos['rm_id'] == rm_id) &
                (daily_pos['date'] >= start_date) &
                (daily_pos['date'] <= end_date)
            )
            row['po_in_window'] = daily_pos.loc[po_window_mask, 'po_quantity'].sum()
            
            start_ly = start_date - timedelta(days=365)
            end_ly = end_date - timedelta(days=365)
            rec_ly_mask = (
                (daily_receivals['rm_id'] == rm_id) &
                (daily_receivals['date'] >= start_ly) &
                (daily_receivals['date'] <= end_ly)
            )
            row['same_period_last_year'] = daily_receivals.loc[rec_ly_mask, 'weight_sum'].sum()
            
            training_data_list.append(row)
            sample_weights_list.append(year_weight)  # ← WEIGHT per sample
            year_total += 1
    
    year_totals[year] = year_total
    print(f"    Year {year}: {year_total} samples")
    gc.collect()

train_df = pd.DataFrame(training_data_list)
sample_weights = np.array(sample_weights_list)

# Normalize weights to sum to 1 (oppure no, a seconda della libreria)
sample_weights = sample_weights / sample_weights.mean()  # Normalization

print(f"\n✅ Training dataset with temporal weighting: {train_df.shape}")
print(f"Sample weight distribution:")
print(f"  Min weight: {sample_weights.min():.3f}")
print(f"  Max weight: {sample_weights.max():.3f}")
print(f"  Mean weight: {sample_weights.mean():.3f}")

print(f"\nBreakdown by year:")
for year in TRAINING_YEARS:
    count = year_totals.get(year, 0)
    pct = (count / len(train_df)) * 100 if len(train_df) > 0 else 0
    total_weight = sample_weights[year_totals[year]:year_totals[year] + count].sum() if count > 0 else 0
    print(f"  {year}: {count:6d} samples ({pct:5.1f}%) | Total weight: {total_weight:8.1f}")

del training_data_list, sample_weights_list
gc.collect()

🔄 Generating training dataset with temporal weighting...
Temporal weights:
  2016: 0.3x
  2017: 0.4x
  2018: 0.6x
  2019: 0.8x
  2020: 1.0x
  2021: 1.3x
  2022: 1.6x
  2023: 1.9x
  2024: 2.5x

Using 120 materials from years [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
  Processing year 2016 (weight=0.3x)...
    Year 2016: 260 samples
  Processing year 2017 (weight=0.4x)...
    Year 2017: 266 samples
  Processing year 2018 (weight=0.6x)...
    Year 2018: 239 samples
  Processing year 2019 (weight=0.8x)...
    Year 2019: 281 samples
  Processing year 2020 (weight=1.0x)...
    Year 2020: 238 samples
  Processing year 2021 (weight=1.3x)...
    Year 2021: 306 samples
  Processing year 2022 (weight=1.6x)...
    Year 2022: 330 samples
  Processing year 2023 (weight=1.9x)...
    Year 2023: 329 samples
  Processing year 2024 (weight=2.5x)...
    Year 2024: 360 samples

✅ Training dataset with temporal weighting: (2609, 14)
Sample weight distribution:
  Min weight: 0.241
  Max weight: 

22

In [26]:
# %% [markdown]
# ## 6.5 Advanced Feature Engineering

# %%
print("🔧 Adding advanced features...")

# Per ogni campione, calcola features avanzate
def add_advanced_features(train_df, daily_receivals, daily_pos):
    """Add volatility, MA, lag features"""
    
    train_df = train_df.copy()
    
    advanced_features = []
    
    for idx, row in train_df.iterrows():
        rm_id = row['rm_id']
        ref_date = row['start_date']
        window_days = row['window_days']
        
        # 1. VOLATILITY (std dev dei 30 giorni precedenti)
        rec_hist = daily_receivals[
            (daily_receivals['rm_id'] == rm_id) &
            (daily_receivals['date'] >= ref_date - timedelta(days=30)) &
            (daily_receivals['date'] < ref_date)
        ]
        volatility = rec_hist['weight_sum'].std() if len(rec_hist) > 1 else 0
        
        # 2. MOVING AVERAGES
        ma_7 = rec_hist['weight_sum'].tail(7).mean() if len(rec_hist) >= 7 else 0
        ma_30 = rec_hist['weight_sum'].mean()
        
        # 3. LAG FEATURES (deliveries 60 giorni fa)
        lag_60_start = ref_date - timedelta(days=120)
        lag_60_end = ref_date - timedelta(days=60)
        lag_60_data = daily_receivals[
            (daily_receivals['rm_id'] == rm_id) &
            (daily_receivals['date'] >= lag_60_start) &
            (daily_receivals['date'] <= lag_60_end)
        ]
        lag_60 = lag_60_data['weight_sum'].sum()
        
        advanced_features.append({
            'volatility_30d': volatility,
            'ma_7d': ma_7,
            'ma_30d': ma_30,
            'lag_60d': lag_60
        })
    
    adv_df = pd.DataFrame(advanced_features)
    for col in adv_df.columns:
        train_df[col] = adv_df[col]
    
    return train_df

train_df = add_advanced_features(train_df, daily_receivals, daily_pos)
print(f"✅ Advanced features added")
print(f"   Features: {train_df.columns[-4:]}")


🔧 Adding advanced features...
✅ Advanced features added
   Features: Index(['volatility_30d', 'ma_7d', 'ma_30d', 'lag_60d'], dtype='object')


## 9. Prepare Training and Validation Sets (WITH LOG TARGET)

In [27]:
# %% [markdown]
# ## 9. Prepare Training and Validation Sets

# %%
print("📊 Creating PROPER time-series split...")

feature_cols = [
    'rm_id', 'window_days', 'end_month', 'end_quarter', 
    'end_dayofyear', 'is_q1',
    'rec_hist_30d_sum', 'rec_hist_30d_mean',
    'po_hist_30d_sum', 'po_in_window', 
    'same_period_last_year',
    'volatility_30d', 'ma_7d', 'ma_30d', 'lag_60d'
]

# Split RIGIDO per anno
train_mask = train_df['end_date'].dt.year < 2024
val_mask = train_df['end_date'].dt.year == 2024

X_train = train_df[train_mask].copy()
X_val = train_df[val_mask].copy()

# ← USA TARGET ORIGINALE (NOT LOG)
y_train = X_train['target']  # ← ORIGINAL SCALE
y_val = X_val['target']      # ← ORIGINAL SCALE

# Sample weights
train_indices = X_train.index.tolist()
val_indices = X_val.index.tolist()

sample_weights_train = sample_weights[train_indices]
sample_weights_val = sample_weights[val_indices]

X_train_features = X_train[feature_cols].copy()
X_val_features = X_val[feature_cols].copy()

X_train_features['rm_id'] = X_train_features['rm_id'].astype(float)
X_val_features['rm_id'] = X_val_features['rm_id'].astype(float)

print(f"✅ Time-Series Split (ORIGINAL SCALE):")
print(f"   Training: {X_train_features.shape[0]} samples")
print(f"   Validation: {X_val_features.shape[0]} samples")
print(f"   y_train range: {y_train.min():.0f} to {y_train.max():.0f} kg")


📊 Creating PROPER time-series split...
✅ Time-Series Split (ORIGINAL SCALE):
   Training: 2249 samples
   Validation: 360 samples
   y_train range: 143 to 14742968 kg


## 10. Train LightGBM Model with Temporal Sample Weights

In [28]:
# %% [markdown]
# ## 10. Train LightGBM (STRONGER Regularization)

# %%
from lightgbm import LGBMRegressor
import lightgbm as lgb

print("🚀 Training LightGBM with STRONGER regularization...")

model = LGBMRegressor(
    objective='quantile',
    alpha=0.2,
    n_estimators=1500,  # Ridotto da 2000
    learning_rate=0.01,  # Ridotto da 0.03 ← CHIAVE
    max_depth=5,  # Ridotto da 8 ← CHIAVE
    num_leaves=32,  # Ridotto da 64 ← CHIAVE
    min_child_samples=50,  # Aumentato da 20 ← CHIAVE
    subsample=0.7,  # Ridotto da 0.8
    colsample_bytree=0.7,  # Ridotto da 0.8
    reg_alpha=1.0,  # Aumentato da 0.1 ← CHIAVE
    reg_lambda=2.0,  # Aumentato da 1.0 ← CHIAVE
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

model.fit(
    X_train_features, y_train,
    sample_weight=sample_weights_train,
    eval_set=[(X_val_features, y_val)],
    eval_sample_weight=[sample_weights_val],
    eval_metric='quantile',
    callbacks=[
        lgb.early_stopping(stopping_rounds=150, verbose=True),
        lgb.log_evaluation(period=100)
    ]
)

print(f"✅ Training completed with STRONG regularization")


🚀 Training LightGBM with STRONGER regularization...
Training until validation scores don't improve for 150 rounds
[100]	valid_0's quantile: 85275
[200]	valid_0's quantile: 76820.8
[300]	valid_0's quantile: 69377.1
[400]	valid_0's quantile: 64716.6
[500]	valid_0's quantile: 67685
Early stopping, best iteration is:
[424]	valid_0's quantile: 64507.6
✅ Training completed with STRONG regularization


## 10.5 Train CatBoost for Comparison

In [30]:
# %%
from catboost import CatBoostRegressor
import catboost as cat

print("🚀 Training CatBoost for comparison...")

catboost_model = CatBoostRegressor(
    loss_function='Quantile:alpha=0.2',  # Quantile 0.2
    iterations=1500,
    learning_rate=0.01,
    depth=5,
    l2_leaf_reg=2.0,
    random_state=42,
    verbose=False,
    thread_count=-1
)

catboost_model.fit(
    X_train_features, y_train,
    sample_weight=sample_weights_train,
    eval_set=(X_val_features, y_val),
    early_stopping_rounds=150,
    verbose=100
)

# Predict on validation
cat_pred_val = catboost_model.predict(X_val_features)
cat_mae = np.mean(np.abs(cat_pred_val - y_val))

# Compare with LightGBM
lgb_pred_val = model.predict(X_val_features)
lgb_mae = np.mean(np.abs(lgb_pred_val - y_val))

print(f"\n📊 Model Comparison (Validation):")
print(f"{'='*50}")
print(f"LightGBM MAE: {lgb_mae:>12,.0f} kg")
print(f"CatBoost MAE: {cat_mae:>12,.0f} kg")
print(f"Difference:   {abs(cat_mae - lgb_mae):>12,.0f} kg")

if cat_mae < lgb_mae:
    print(f"✅ CatBoost is {((lgb_mae - cat_mae) / lgb_mae * 100):.1f}% BETTER")
    print(f"   → Use CatBoost for final submission")
    use_catboost = True
else:
    print(f"✅ LightGBM is {((cat_mae - lgb_mae) / lgb_mae * 100):.1f}% BETTER")
    print(f"   → Keep LightGBM")
    use_catboost = False


🚀 Training CatBoost for comparison...
0:	learn: 117015.1353223	test: 92071.6135579	best: 92071.6135579 (0)	total: 48ms	remaining: 1m 12s
100:	learn: 101010.9813686	test: 81952.2731676	best: 81952.2731676 (100)	total: 118ms	remaining: 1.64s
200:	learn: 90667.7266279	test: 75620.3542624	best: 75620.3542624 (200)	total: 193ms	remaining: 1.25s
300:	learn: 78274.0430699	test: 69202.3355396	best: 69202.3355396 (300)	total: 273ms	remaining: 1.09s
400:	learn: 63366.3479273	test: 70935.4861958	best: 68248.5433871 (331)	total: 352ms	remaining: 964ms
Stopped by overfitting detector  (150 iterations wait)

bestTest = 68248.54339
bestIteration = 331

Shrink model to first 332 iterations.

📊 Model Comparison (Validation):
LightGBM MAE:      288,526 kg
CatBoost MAE:      307,022 kg
Difference:         18,496 kg
✅ LightGBM is 6.4% BETTER
   → Keep LightGBM


## 11. Feature Importance Analysis

In [31]:
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n📈 Top 15 Most Important Features:")
print(feature_importance.head(15).to_string(index=False))


📈 Top 15 Most Important Features:
              feature  importance
         po_in_window        1134
              lag_60d         902
same_period_last_year         649
                rm_id         643
          window_days         572
     rec_hist_30d_sum         571
    rec_hist_30d_mean         485
               ma_30d         353
      po_hist_30d_sum         331
                ma_7d         245
       volatility_30d         173
        end_dayofyear         139
            end_month         102
          end_quarter           6
                is_q1           1


## 12. Refit on Full Dataset (WITH LOG TARGET)

In [32]:
print("🔄 Refitting model on full training data...")

# Combine train + validation
X_full = pd.concat([X_train_features, X_val_features])
y_full = pd.concat([y_train, y_val])  # ← Already in LOG space

# Combine weights
sample_weights_full = np.concatenate([sample_weights_train, sample_weights_val])

# Use best_iteration from previous training
final_model = LGBMRegressor(
    objective='quantile',
    alpha=0.2,
    n_estimators=model.best_iteration_ if model.best_iteration_ > 0 else 500,
    learning_rate=0.01,  # ← Keep strong regularization
    num_leaves=32,
    max_depth=5,
    min_child_samples=50,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_alpha=1.0,
    reg_lambda=2.0,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

final_model.fit(X_full, y_full, sample_weight=sample_weights_full)

print("✅ Final model trained on full dataset (LOG SCALE)")


🔄 Refitting model on full training data...
✅ Final model trained on full dataset (LOG SCALE)


## 13. Prepare Test Features

In [33]:
print("🔧 Creating test features...")

test_df = prediction_mapping.copy()

# Remove timezone
test_df['forecast_start_date'] = pd.to_datetime(test_df['forecast_start_date']).dt.tz_localize(None)
test_df['forecast_end_date'] = pd.to_datetime(test_df['forecast_end_date']).dt.tz_localize(None)

# Basic date features
test_df['window_days'] = (test_df['forecast_end_date'] - test_df['forecast_start_date']).dt.days + 1
test_df['end_month'] = test_df['forecast_end_date'].dt.month
test_df['end_quarter'] = test_df['forecast_end_date'].dt.quarter
test_df['end_dayofyear'] = test_df['forecast_end_date'].dt.dayofyear
test_df['is_q1'] = (test_df['end_quarter'] == 1).astype(int)

# Historical features (30d)
print("  Computing historical features (30d)...")

hist_features_30d = []

for _, row in test_df.iterrows():
    ref_date = row['forecast_start_date']
    rm_id = row['rm_id']
    
    rec_stats = calculate_historical_stats(daily_receivals, ref_date, rm_id, 30)
    
    po_mask = (
        (daily_pos['rm_id'] == rm_id) &
        (daily_pos['date'] >= ref_date - timedelta(days=30)) &
        (daily_pos['date'] < ref_date)
    )
    po_sum = daily_pos.loc[po_mask, 'po_quantity'].sum()
    
    hist_features_30d.append({
        'rec_hist_30d_sum': rec_stats['sum'],
        'rec_hist_30d_mean': rec_stats['mean'],
        'po_hist_30d_sum': po_sum
    })

hist_df = pd.DataFrame(hist_features_30d)
for col in hist_df.columns:
    test_df[col] = hist_df[col]

# PO in window
print("  Computing POs in test windows...")
po_in_window_test = []

for _, row in test_df.iterrows():
    mask = (
        (daily_pos['rm_id'] == row['rm_id']) &
        (daily_pos['date'] >= row['forecast_start_date']) &
        (daily_pos['date'] <= row['forecast_end_date'])
    )
    po_in_window_test.append(daily_pos.loc[mask, 'po_quantity'].sum())

test_df['po_in_window'] = po_in_window_test

# Year-over-year
print("  Computing year-over-year...")
yoy_test = []

for _, row in test_df.iterrows():
    start_ly = row['forecast_start_date'] - timedelta(days=365)
    end_ly = row['forecast_end_date'] - timedelta(days=365)
    
    mask = (
        (daily_receivals['rm_id'] == row['rm_id']) &
        (daily_receivals['date'] >= start_ly) &
        (daily_receivals['date'] <= end_ly)
    )
    
    yoy_test.append(daily_receivals.loc[mask, 'weight_sum'].sum())

test_df['same_period_last_year'] = yoy_test

# ← ADD ADVANCED FEATURES (SAME AS TRAINING)
print("  Computing advanced features...")

advanced_features_test = []

for idx, row in test_df.iterrows():
    rm_id = row['rm_id']
    ref_date = row['forecast_start_date']
    
    # VOLATILITY
    rec_hist = daily_receivals[
        (daily_receivals['rm_id'] == rm_id) &
        (daily_receivals['date'] >= ref_date - timedelta(days=30)) &
        (daily_receivals['date'] < ref_date)
    ]
    volatility = rec_hist['weight_sum'].std() if len(rec_hist) > 1 else 0
    
    # MOVING AVERAGES
    ma_7 = rec_hist['weight_sum'].tail(7).mean() if len(rec_hist) >= 7 else 0
    ma_30 = rec_hist['weight_sum'].mean()
    
    # LAG FEATURES
    lag_60_start = ref_date - timedelta(days=120)
    lag_60_end = ref_date - timedelta(days=60)
    lag_60_data = daily_receivals[
        (daily_receivals['rm_id'] == rm_id) &
        (daily_receivals['date'] >= lag_60_start) &
        (daily_receivals['date'] <= lag_60_end)
    ]
    lag_60 = lag_60_data['weight_sum'].sum()
    
    advanced_features_test.append({
        'volatility_30d': volatility,
        'ma_7d': ma_7,
        'ma_30d': ma_30,
        'lag_60d': lag_60
    })

adv_df_test = pd.DataFrame(advanced_features_test)
for col in adv_df_test.columns:
    test_df[col] = adv_df_test[col]

# ← USE SAME feature_cols AS TRAINING (15 features)
feature_cols_test = [
    'rm_id', 'window_days', 'end_month', 'end_quarter', 
    'end_dayofyear', 'is_q1',
    'rec_hist_30d_sum', 'rec_hist_30d_mean',
    'po_hist_30d_sum', 'po_in_window', 
    'same_period_last_year',
    'volatility_30d', 'ma_7d', 'ma_30d', 'lag_60d'  # ← 15 TOTAL
]

X_test = test_df[feature_cols_test].copy()
X_test['rm_id'] = X_test['rm_id'].astype(float)

print(f"✅ Test features ready: {X_test.shape}")
print(f"   Features: {len(feature_cols_test)} (same as training)")


🔧 Creating test features...
  Computing historical features (30d)...
  Computing POs in test windows...
  Computing year-over-year...
  Computing advanced features...
✅ Test features ready: (30450, 15)
   Features: 15 (same as training)


## 14. Generate Predictions (WITH INVERSE LOG)

In [34]:
# %% [markdown]
# ## 14. Generate Predictions

# %%
print("🎯 Generating predictions...")

# Predict (ORIGINAL SCALE - NO log)
predictions = final_model.predict(X_test)
predictions = np.maximum(predictions, 0)  # No negative

submission = pd.DataFrame({
    'ID': test_df['ID'],
    'predicted_weight': predictions,
    'rm_id': test_df['rm_id'],
    'forecast_end_date': test_df['forecast_end_date']
})

print(f"✅ Generated {len(submission)} predictions")
print(f"   Prediction stats (original scale):")
print(f"   - min={predictions.min():.2f} kg")
print(f"   - max={predictions.max():.2f} kg")
print(f"   - mean={predictions.mean():.2f} kg")

🎯 Generating predictions...
✅ Generated 30450 predictions
   Prediction stats (original scale):
   - min=1317.61 kg
   - max=1814707.80 kg
   - mean=59857.73 kg


## 15. Enforce Monotonicity (Cumulative Property)

In [35]:
print("🔧 Enforcing cumulative monotonicity (CORRECTED)...")

# Create working copy with metadata from test_df DIRECTLY (NOT submission)
submission_work = pd.DataFrame({
    'ID': test_df['ID'].values,
    'rm_id': test_df['rm_id'].values,
    'forecast_start_date': test_df['forecast_start_date'].values,
    'forecast_end_date': test_df['forecast_end_date'].values,
    'predicted_weight': predictions
})

print(f"Before monotonicity: min={submission_work['predicted_weight'].min():.2f}, max={submission_work['predicted_weight'].max():.2f}")

# Sort by rm_id, then by forecast_end_date for proper cumulative ordering
submission_work = submission_work.sort_values(['rm_id', 'forecast_end_date']).reset_index(drop=True)

# Apply cummax DENTRO OGNI MATERIALE
submission_work['predicted_weight'] = submission_work.groupby('rm_id')['predicted_weight'].cummax()

print(f"After monotonicity: min={submission_work['predicted_weight'].min():.2f}, max={submission_work['predicted_weight'].max():.2f}")

# Sort back by ID for submission format
submission_work = submission_work.sort_values('ID').reset_index(drop=True)

# Final submission
final_submission = submission_work[['ID', 'predicted_weight']].copy()

print(f"✅ Monotonicity enforced")

# Detailed verification
print(f"\n📋 Verification - Sample material progressions:")
for rm_id in submission_work['rm_id'].unique()[:3]:
    mat_subset = submission_work[submission_work['rm_id'] == rm_id].sort_values('ID').head(30)
    is_monotonic = (mat_subset['predicted_weight'].diff().dropna() >= -0.01).all()  # Allow tiny floating point errors
    status = "✅ MONOTONIC" if is_monotonic else "❌ NOT MONOTONIC"
    
    print(f"\n  Material {rm_id}: {status}")
    print(f"    First 10 predictions: {mat_subset['predicted_weight'].head(10).values}")
    print(f"    Min: {mat_subset['predicted_weight'].min():.0f}, Max: {mat_subset['predicted_weight'].max():.0f}")


🔧 Enforcing cumulative monotonicity (CORRECTED)...
Before monotonicity: min=1317.61, max=1814707.80
After monotonicity: min=1709.15, max=1814707.80
✅ Monotonicity enforced

📋 Verification - Sample material progressions:

  Material 365: ✅ MONOTONIC
    First 10 predictions: [4476.2145814  4476.2145814  4476.2145814  4476.2145814  4476.2145814
 4476.2145814  4476.2145814  4656.47810405 4656.47810405 4656.47810405]
    Min: 4476, Max: 11574

  Material 379: ✅ MONOTONIC
    First 10 predictions: [3773.71263 3773.71263 3773.71263 3773.71263 3773.71263 3773.71263
 3773.71263 3773.71263 3773.71263 3773.71263]
    Min: 3774, Max: 3793

  Material 389: ✅ MONOTONIC
    First 10 predictions: [4194.19489016 4194.19489016 4194.19489016 4194.19489016 4194.19489016
 4194.19489016 4194.19489016 4194.19489016 4194.19489016 4194.19489016]
    Min: 4194, Max: 13214


## 16. Save Submission File

In [36]:
output_file = 'gradient_boosting_submission.csv'
final_submission.to_csv(output_file, index=False)

print(f"\n🎉 Submission saved to: {output_file}")
print(f"\n📊 Final Submission Statistics:")
print(final_submission['predicted_weight'].describe())

# Sanity checks
print(f"\n✅ Sanity Checks:")
print(f"  - Total predictions: {len(final_submission)}")
print(f"  - Negative predictions: {(final_submission['predicted_weight'] < 0).sum()}")
print(f"  - Zero predictions: {(final_submission['predicted_weight'] == 0).sum()}")
print(f"  - Missing values: {final_submission['predicted_weight'].isna().sum()}")

# CRITICAL: Verify monotonicity in final submission
print(f"\n🔍 CRITICAL VERIFICATION:")
non_monotonic_count = 0
for rm_id in submission_work['rm_id'].unique():
    mat_data = final_submission.merge(
        submission_work[['ID', 'rm_id', 'forecast_end_date']], 
        on='ID'
    ).sort_values('forecast_end_date')
    mat_data = mat_data[mat_data['rm_id'] == rm_id]
    
    violations = (mat_data['predicted_weight'].diff().dropna() < -0.01).sum()
    non_monotonic_count += violations

print(f"  Monotonicity violations: {non_monotonic_count}")
if non_monotonic_count == 0:
    print(f"  ✅ ALL GOOD - Monotonicity is enforced!")
else:
    print(f"  ❌ PROBLEM - Monotonicity failed for {non_monotonic_count} transitions")



🎉 Submission saved to: gradient_boosting_submission.csv

📊 Final Submission Statistics:
count    3.045000e+04
mean     6.043853e+04
std      2.248198e+05
min      1.709150e+03
25%      3.792847e+03
50%      8.870413e+03
75%      1.387443e+04
max      1.814708e+06
Name: predicted_weight, dtype: float64

✅ Sanity Checks:
  - Total predictions: 30450
  - Negative predictions: 0
  - Zero predictions: 0
  - Missing values: 0

🔍 CRITICAL VERIFICATION:
  Monotonicity violations: 0
  ✅ ALL GOOD - Monotonicity is enforced!


## 17. Historical Validation: Compare with 2024 Actual Data

In [37]:
# Simulate the same prediction windows on 2024 data to validate model performance


print("🔍 Historical Validation: Comparing model predictions with 2024 actuals...")


# Create "submission-like" windows for 2024 (Jan 1 - May 31)
validation_windows = []


# Select top 10 materials for detailed analysis
top_10_materials = daily_receivals.groupby('rm_id').size().sort_values(ascending=False).head(10).index.tolist()


for rm_id in top_10_materials:
    for window_days in [7, 30, 60, 90, 120, 151]:
        start_date = datetime(2024, 1, 1)
        end_date = start_date + timedelta(days=window_days - 1)
        
        # Calculate ACTUAL cumulative weight from historical data
        actual_weight = calculate_window_receivals(
            daily_receivals, start_date, end_date, rm_id
        )
        
        validation_windows.append({
            'rm_id': rm_id,
            'window_days': window_days,
            'end_date': end_date,
            'actual_weight': actual_weight
        })


validation_df = pd.DataFrame(validation_windows)


# Create features for these validation windows (same as test)
validation_df['start_date'] = datetime(2024, 1, 1)
validation_df['end_month'] = validation_df['end_date'].dt.month
validation_df['end_quarter'] = validation_df['end_date'].dt.quarter
validation_df['end_dayofyear'] = validation_df['end_date'].dt.dayofyear
validation_df['is_q1'] = (validation_df['end_quarter'] == 1).astype(int)


# Historical features (30d before 2024-01-01)
ref_date = datetime(2024, 1, 1)
hist_features_val = []


for _, row in validation_df.iterrows():
    rm_id = row['rm_id']
    
    rec_stats = calculate_historical_stats(daily_receivals, ref_date, rm_id, 30)
    
    po_mask = (
        (daily_pos['rm_id'] == rm_id) &
        (daily_pos['date'] >= ref_date - timedelta(days=30)) &
        (daily_pos['date'] < ref_date)
    )
    po_sum = daily_pos.loc[po_mask, 'po_quantity'].sum()
    
    # PO in window
    po_window_mask = (
        (daily_pos['rm_id'] == rm_id) &
        (daily_pos['date'] >= row['start_date']) &
        (daily_pos['date'] <= row['end_date'])
    )
    po_in_win = daily_pos.loc[po_window_mask, 'po_quantity'].sum()
    
    # YoY
    start_ly = row['start_date'] - timedelta(days=365)
    end_ly = row['end_date'] - timedelta(days=365)
    rec_ly_mask = (
        (daily_receivals['rm_id'] == rm_id) &
        (daily_receivals['date'] >= start_ly) &
        (daily_receivals['date'] <= end_ly)
    )
    yoy = daily_receivals.loc[rec_ly_mask, 'weight_sum'].sum()
    
    hist_features_val.append({
        'rec_hist_30d_sum': rec_stats['sum'],
        'rec_hist_30d_mean': rec_stats['mean'],
        'po_hist_30d_sum': po_sum,
        'po_in_window': po_in_win,
        'same_period_last_year': yoy
    })


hist_val_df = pd.DataFrame(hist_features_val)
for col in hist_val_df.columns:
    validation_df[col] = hist_val_df[col]


# =========================================================================
# ← FIX: ADD ADVANCED FEATURES (SAME AS TEST SECTION - Section 13)
# =========================================================================

print("  Computing advanced features...")

advanced_features_validation = []

for idx, row in validation_df.iterrows():
    rm_id = row['rm_id']
    ref_date = row['start_date']
    
    # VOLATILITY (std dev dei 30 giorni precedenti)
    rec_hist = daily_receivals[
        (daily_receivals['rm_id'] == rm_id) &
        (daily_receivals['date'] >= ref_date - timedelta(days=30)) &
        (daily_receivals['date'] < ref_date)
    ]
    volatility = rec_hist['weight_sum'].std() if len(rec_hist) > 1 else 0
    
    # MOVING AVERAGES
    ma_7 = rec_hist['weight_sum'].tail(7).mean() if len(rec_hist) >= 7 else 0
    ma_30 = rec_hist['weight_sum'].mean()
    
    # LAG FEATURES (deliveries 60 giorni fa)
    lag_60_start = ref_date - timedelta(days=120)
    lag_60_end = ref_date - timedelta(days=60)
    lag_60_data = daily_receivals[
        (daily_receivals['rm_id'] == rm_id) &
        (daily_receivals['date'] >= lag_60_start) &
        (daily_receivals['date'] <= lag_60_end)
    ]
    lag_60 = lag_60_data['weight_sum'].sum()
    
    advanced_features_validation.append({
        'volatility_30d': volatility,
        'ma_7d': ma_7,
        'ma_30d': ma_30,
        'lag_60d': lag_60
    })

adv_df_validation = pd.DataFrame(advanced_features_validation)
for col in adv_df_validation.columns:
    validation_df[col] = adv_df_validation[col]

print(f"✅ Advanced features added")

# =========================================================================


# Prepare features (NOW ALL 15 COLUMNS ARE PRESENT!)
X_val_check = validation_df[feature_cols_test].copy()
X_val_check['rm_id'] = X_val_check['rm_id'].astype(float)


# Predict
validation_df['predicted_weight'] = final_model.predict(X_val_check)
validation_df['predicted_weight'] = validation_df['predicted_weight'].clip(lower=0)


# Calculate metrics
validation_df['error'] = validation_df['predicted_weight'] - validation_df['actual_weight']
validation_df['abs_error'] = validation_df['error'].abs()
validation_df['pct_error'] = (validation_df['error'] / validation_df['actual_weight'].replace(0, 1)) * 100


print(f"\n📊 Historical Validation Results (2024 Data):")
print(f"{'='*60}")
print(f"Sample size: {len(validation_df)} windows from top 10 materials")
print(f"\n{'Metric':<30} {'Value':>15}")
print(f"{'-'*60}")
print(f"{'Mean Actual Weight':<30} {validation_df['actual_weight'].mean():>15,.0f} kg")
print(f"{'Mean Predicted Weight':<30} {validation_df['predicted_weight'].mean():>15,.0f} kg")
print(f"{'Median Actual Weight':<30} {validation_df['actual_weight'].median():>15,.0f} kg")
print(f"{'Median Predicted Weight':<30} {validation_df['predicted_weight'].median():>15,.0f} kg")
print(f"\n{'Mean Absolute Error (MAE)':<30} {validation_df['abs_error'].mean():>15,.0f} kg")
print(f"{'Median Absolute Error':<30} {validation_df['abs_error'].median():>15,.0f} kg")
print(f"{'Mean Percentage Error':<30} {validation_df['pct_error'].mean():>15.1f}%")


# Detailed breakdown by window length
print(f"\n📈 Performance by Window Length:")
print(f"{'='*60}")
for window in [7, 30, 60, 90, 120, 151]:
    subset = validation_df[validation_df['window_days'] == window]
    if len(subset) > 0:
        mae = subset['abs_error'].mean()
        print(f"  {window:3d} days: MAE = {mae:>12,.0f} kg (avg actual: {subset['actual_weight'].mean():>12,.0f} kg)")


# Show examples
print(f"\n🔍 Sample Predictions (First 10 rows):")
print(validation_df[['rm_id', 'window_days', 'actual_weight', 'predicted_weight', 'error']].head(10).to_string(index=False))


print(f"\n✅ Validation complete! Check if predictions align with 2024 actuals.")

🔍 Historical Validation: Comparing model predictions with 2024 actuals...
  Computing advanced features...
✅ Advanced features added

📊 Historical Validation Results (2024 Data):
Sample size: 60 windows from top 10 materials

Metric                                   Value
------------------------------------------------------------
Mean Actual Weight                     294,663 kg
Mean Predicted Weight                  215,241 kg
Median Actual Weight                   103,286 kg
Median Predicted Weight                 74,228 kg

Mean Absolute Error (MAE)               98,402 kg
Median Absolute Error                   23,212 kg
Mean Percentage Error                 354126.8%

📈 Performance by Window Length:
    7 days: MAE =       10,391 kg (avg actual:            0 kg)
   30 days: MAE =       38,259 kg (avg actual:       80,382 kg)
   60 days: MAE =       34,043 kg (avg actual:      186,854 kg)
   90 days: MAE =       16,573 kg (avg actual:      304,732 kg)
  120 days: MAE =      178,3

## 17.5 Refined Validation: Only Active Materials in 2024

In [38]:
print("🔍 Refined Validation: Only materials ACTIVE in 2024...\n")

# Filter: keep only rows where actual_weight > 0 (has real 2024 data)
validation_active = validation_df[validation_df['actual_weight'] > 0].copy()

print(f"Active windows in 2024: {len(validation_active)} out of {len(validation_df)}")
print(f"Inactive materials (0 actual weight): {len(validation_df) - len(validation_active)}")

if len(validation_active) > 0:
    # Recalculate metrics ONLY for active windows
    validation_active['error'] = (validation_active['predicted_weight'] - 
                                  validation_active['actual_weight'])
    validation_active['abs_error'] = validation_active['error'].abs()
    validation_active['pct_error'] = (validation_active['error'] / 
                                      validation_active['actual_weight']) * 100
    
    print(f"\n📊 Refined Results (Only Active Materials):")
    print(f"{'='*65}")
    print(f"{'Metric':<35} {'Value':>25}")
    print(f"{'-'*65}")
    print(f"{'Mean Actual Weight':<35} {validation_active['actual_weight'].mean():>25,.0f} kg")
    print(f"{'Mean Predicted Weight':<35} {validation_active['predicted_weight'].mean():>25,.0f} kg")
    print(f"{'Median Actual Weight':<35} {validation_active['actual_weight'].median():>25,.0f} kg")
    print(f"{'Median Predicted Weight':<35} {validation_active['predicted_weight'].median():>25,.0f} kg")
    print(f"\n{'Mean Absolute Error (MAE)':<35} {validation_active['abs_error'].mean():>25,.0f} kg")
    print(f"{'Median Absolute Error':<35} {validation_active['abs_error'].median():>25,.0f} kg")
    print(f"{'Mean Absolute % Error (MAPE)':<35} {validation_active['pct_error'].abs().mean():>25.1f}%")
    print(f"{'Median Absolute % Error':<35} {validation_active['pct_error'].abs().median():>25.1f}%")
    
    # Breakdown by window
    print(f"\n📈 Performance by Window Length (Active Only):")
    print(f"{'='*65}")
    for window in [7, 30, 60, 90, 120, 151]:
        subset = validation_active[validation_active['window_days'] == window]
        if len(subset) > 0:
            mae = subset['abs_error'].mean()
            mape = subset['pct_error'].abs().mean()
            print(f"  {window:3d} days: {len(subset):2d} windows | MAE={mae:>10,.0f} kg | MAPE={mape:>6.1f}%")
        else:
            print(f"  {window:3d} days: No active data in 2024")
    
    # Show best and worst predictions
    print(f"\n🎯 Best Predictions (Lowest Error):")
    best = validation_active.nsmallest(5, 'abs_error')[['rm_id', 'window_days', 'actual_weight', 'predicted_weight', 'pct_error']]
    print(best.to_string(index=False))
    
    print(f"\n⚠️ Worst Predictions (Highest Error):")
    worst = validation_active.nlargest(5, 'abs_error')[['rm_id', 'window_days', 'actual_weight', 'predicted_weight', 'pct_error']]
    print(worst.to_string(index=False))
    
    # Overall assessment
    mape_mean = validation_active['pct_error'].abs().mean()
    if mape_mean < 20:
        assessment = "✅ EXCELLENT - Model is very accurate"
    elif mape_mean < 50:
        assessment = "✅ GOOD - Model predictions are reliable"
    elif mape_mean < 100:
        assessment = "⚠️ FAIR - Model predictions are acceptable"
    else:
        assessment = "❌ POOR - Model predictions need improvement"
    
    print(f"\n{assessment}")
    print(f"Mean MAPE: {mape_mean:.1f}%")
else:
    print("❌ No active materials found in 2024 - cannot validate!")


🔍 Refined Validation: Only materials ACTIVE in 2024...

Active windows in 2024: 39 out of 60
Inactive materials (0 actual weight): 21

📊 Refined Results (Only Active Materials):
Metric                                                  Value
-----------------------------------------------------------------
Mean Actual Weight                                    453,327 kg
Mean Predicted Weight                                 325,692 kg
Median Actual Weight                                  203,255 kg
Median Predicted Weight                               178,871 kg

Mean Absolute Error (MAE)                             145,939 kg
Median Absolute Error                                  30,839 kg
Mean Absolute % Error (MAPE)                             25.3%
Median Absolute % Error                                  22.8%

📈 Performance by Window Length (Active Only):
    7 days: No active data in 2024
   30 days:  7 windows | MAE=    47,458 kg | MAPE=  38.3%
   60 days:  8 windows | MAE=    38,0